In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sn

import healpy as hp
import lsst.sims.maf.metrics as metrics
import lsst.sims.maf.slicers as slicers
import lsst.sims.maf.metricBundles as metricBundles
import lsst.sims.maf.db as db


In [ ]:
# connect to opsim database
opsdb_baseline = db.OpsimDatabase('opsdb/baseline2018a.db')
opsdb_pontus = db.OpsimDatabase('opsdb/pontus_2573.db')
opsdb_baseline10yrs = db.OpsimDatabase('opsdb/baseline_2snap_v1.3_10yrs.db')

# output directory
outDir = 'outdir'
resultsDb = db.ResultsDb(outDir=outDir)

In [ ]:
class timeGapsMetric(metrics.BaseMetric):
    """
    returns all time gaps between two filters
    """

    def __init__(self, colname=['observationStartMJD', 'filter', 'fiveSigmaDepth'], 
                 filename='dT.pkl', fltpair=['y','i'],
                 dataout=True, **kwargs):
        self.colname = colname
        self.filename = filename
        self.fltpair = fltpair
        self.dataout = dataout
        
        #if os.path.isfile(filename):
            # rm old file
        #    os.system("rm filename")
        
        if self.dataout:
            super().__init__(col=self.colname, metricDtype='object', **kwargs)
        else:
            super().__init__(col=self.colname, metricDtype='float', **kwargs)
        
    def dT(self, dataSlice, f0='i', f1='r'):
        ''' return an array that contains all time gaps between two filters'''
        idx0 = dataSlice['filter'] == f0
        idx1 = dataSlice['filter'] == f1
        
        timeCol0 = dataSlice['observationStartMJD'][idx0]
        timeCol1 = dataSlice['observationStartMJD'][idx1]

        timeCol0 = timeCol0.reshape((len(timeCol0), 1))
        timeCol1 = timeCol1.reshape((len(timeCol1), 1))

        diffmat = np.abs( np.subtract(timeCol0, timeCol1.T) )

        return diffmat.flatten()


    def load_from_pkl(self, filename="test_pkl.pkl"):
        '''load dataframe from pickle'''
        if os.path.isfile(filename):
            df = pd.read_pickle(filename)
        else:
            df = pd.DataFrame()
            df.to_pickle(filename)
        return df
    
    def save_to_file(self, dic, filename="test_pkl.pkl"):
        '''save dict item to pickle file'''
        
        df = self.load_from_pkl(filename)

        df = df.append(pd.DataFrame(dic), ignore_index=True)

        df.to_pickle(filename)
    
    def run(self, dataSlice, slicePoint=None):
        # sort dataSlice
        flt = ['u', 'g', 'r', 'i', 'z', 'y']
        fdict = {'u':0, 'g':1, 'r':2, 'i':3, 'z':4, 'y':5}

        dataSlice.sort(order='observationStartMJD')

        dT = self.dT(dataSlice, f0=self.fltpair[0], f1=self.fltpair[1])
        #print(type(dataSlice['fieldRA']), dataSlice['fieldDec'])
        #dT_list = []
        #for i in range(len(dataSlice['fieldRA'])):
        #   dT_list.append(dT)
        
        dic = {'ra':np.mean(dataSlice['fieldRA']), 'dec':np.mean(dataSlice['fieldDec']), 'dT': [dT]}
        # print(dic)
        # save to file
        self.save_to_file(dic, self.filename)
        
        # return dT
        if self.dataout:
            result = dT
            return result
        else:
            f0 = self.fltpair[0]
            f1 = self.fltpair[1]
            result = np.min(dT) if len(dT)!=0 else np.inf
            return float(result)
      

In [ ]:
# run the metric, check same filename in folder
metric = timeGapsMetric(colname=['observationStartMJD', 'filter', 'fiveSigmaDepth'], 
                          filename='dT_ri.pkl', fltpair=['r','i'], dataout=False)
slicer = slicers.HealpixSlicer(nside=16)

sqlconstraint = 'night<400'
metricSky = metricBundles.MetricBundle(metric,slicer,sqlconstraint)

group = metricBundles.MetricBundleGroup({'metricSky':metricSky}, opsdb_pontus, outDir=outDir, resultsDb=resultsDb)
group.runAll()

In [ ]:
# run for all filter pairs

slicer = slicers.HealpixSlicer(nside=16)

sqlconstraint = 'night<400'

# create an dict to run metric for all pairs
metricSkyDict = {}
flt = ['u', 'g', 'r', 'i', 'z', 'y']
for i, f0 in enumerate(flt):
    for f1 in flt[i:]:
        filename = 'dT_{}{}_pontus.pkl'.format(f0, f1)
        fltpair = [f0, f1]
        metric = timeGapsMetric(colname=['observationStartMJD', 'filter', 'fiveSigmaDepth'], 
                          filename=filename, fltpair=fltpair, dataout=False)
        
        metricSky = metricBundles.MetricBundle(metric,slicer,sqlconstraint)
        metricSkyname = 'metricSky_{}{}'.format(f0, f1)
        metricSkyDict[metricSkyname] = metricSky

In [ ]:
group = metricBundles.MetricBundleGroup(metricSkyDict, opsdb_pontus, outDir=outDir, resultsDb=resultsDb)
group.runAll()


In [ ]:
# plot all histogram

fig, axs = plt.subplots(6, 6, figsize=(24, 24)); # 6 axes on a 2x3 grid

flt = ['u', 'g', 'r', 'i', 'z', 'y']
fdict = {'u':0, 'g':1, 'r':2, 'i':3, 'z':4, 'y':5}
tlim = 1.5 # in hours
for i, f0 in enumerate(flt):
    for f1 in flt[i:]:
        dT = np.concatenate(df.dT)
        filename = 'dT_{}{}_pontus.pkl'.format(f0, f1)
        df = pd.read_pickle(filename)
        dT = np.concatenate(df.dT)
        # dT = dT[dT!=0]
        #axs[fdict[f0], fdict[f1]].hist(dT[dT<1/24],bins=100); 
        if f0==f1:
            axs[fdict[f0], fdict[f1]].hist(dT[dT<tlim/24]*24,bins=100); 
            axs[fdict[f0], fdict[f1]].set_xlabel(f0+f1)
        else:
            axs[fdict[f0], fdict[f1]].axis('off')
            axs[fdict[f1], fdict[f0]].hist(dT[dT<tlim/24]*24,bins=100);        
            axs[fdict[f1], fdict[f0]].set_xlabel(f0+f1)

In [ ]:
# plot skymap 

fig, axs = plt.subplots(6, 6, figsize=(24, 24), 
                        subplot_kw={'projection': 'mollweide'}); # 6 axes on a 2x3 grid

for i, f0 in enumerate(flt):
    for f1 in flt[i:]:
    
    plot_mwd(axs[1,1], df.ra.values, df.dec.values, df['dT'].apply(get_tmin).values)